## Importing dependencies

In [1]:
import numpy as np
import cv2
from scipy.spatial import distance
import random
import os

## Functions

In [2]:
def recoverTransformation(R_Points, M_Points):
    A = np.array([[R_Points[0][0][1],R_Points[0][1][1],1,0,0,0],[0,0,0,R_Points[0][0][1],R_Points[0][1][1],1],
                  [R_Points[1][0][1],R_Points[1][1][1],1,0,0,0],[0,0,0,R_Points[1][0][1],R_Points[1][1][1],1],
                  [R_Points[2][0][1],R_Points[2][1][1],1,0,0,0],[0,0,0,R_Points[2][0][1],R_Points[2][1][1],1]])
    
    B = np.array([[R_Points[0][0][0]],[R_Points[0][1][0]],[R_Points[1][0][0]],[R_Points[1][1][0]],[R_Points[2][0][0]],[R_Points[2][1][0]]])
    
    Transform = np.matmul(np.linalg.pinv(A),B)
    Transform = Transform.reshape((2, 3))
    x = []
    for i in range(M_Points.shape[0]):
        x.append([M_Points[i][0][1], M_Points[i][1][1], 1])
    x = np.array(x)
    
    Transform_Pts = np.matmul(Transform, np.transpose(x))
    inliers = []
    for i in range(M_Points.shape[0]):
        MSE = ((M_Points[i][1][0] - Transform_Pts[1][i])**2 + (M_Points[i][0][0] - Transform_Pts[0][i])**2)**0.5
        if (MSE < 3):
            inliers.append(M_Points[i])
    
    inliers = np.array(inliers)
    
    return [Transform, inliers]

In [3]:
def extract_features(imagearray, num_points):
    image = imagearray
    sift = cv2.SIFT_create(num_points)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

In [4]:
def stitchImages(I1, I2):
    N = 400
    print("I1.dtype" , I1.dtype)
    [feature1, descriptor1] = extract_features(I1, N)
    [feature2, descriptor2] = extract_features(I2, N)

    f1 = np.zeros((4, N))
    f2 = np.zeros((4, N))
    
    for i in range(N):
        f1[0][i], f1[1][i] = feature1[i].pt
        f1[2][i] = feature1[i].size
        f1[3][i] = feature1[i].angle
        f2[0][i], f2[1][i] = feature2[i].pt
        f2[2][i] = feature2[i].size
        f2[3][i] = feature2[i].angle
    
    rdf1 = random.sample(feature1, 200)
    rdf2 = random.sample(feature2, 200)
    
    featurePoints1 = cv2.drawKeypoints(I1, rdf1, np.array([]), (0,255,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    featurePoints2 = cv2.drawKeypoints(I2, rdf2, np.array([]), (0,255,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    distances = distance.cdist(descriptor1, descriptor2, 'euclidean')
    
    matchingPts1 = []
    matchingPts2 = []
    
    threshold = np.median(distances) * 0.7  # Adjust the scaling factor as needed

    f1 = np.transpose(f1)
    f2 = np.transpose(f2)
    
    for i in range(distances.shape[0]):
        for j in range(distances.shape[1]):
            if(distances[i][j] < threshold):
                matchingPts1.append(f1[i])
                matchingPts2.append(f2[j])
    
    matchingPts1 = np.array(matchingPts1)
    matchingPts2 = np.array(matchingPts2)
    
    matchingPts = np.transpose(np.array([matchingPts1, matchingPts2]), (1, 2, 0))
    
    matches = np.zeros((I1.shape[0], I1.shape[1] + I2.shape[1], I1.shape[2]), dtype="uint8")
    
    for i in range(I1.shape[0]):
        for j in range(I1.shape[1]):
            matches[i][j] = I1[i][j]
    
    for i in range(I2.shape[0]):
        for j in range(I2.shape[1]):
            matches[i][I1.shape[1]+j] = I2[i][j]
    bMatches = matches.copy()
    
    for i in range(matchingPts.shape[0]):
        matches = cv2.line(matches, (int(matchingPts[i][0][0]), int(matchingPts[i][1][0])), (int(I1.shape[1] + matchingPts[i][0][1]), int(matchingPts[i][1][1])), (0, 255, 255), 1)
    randomPts = np.array(random.sample(list(matchingPts), 4))
    bestInliers = randomPts.copy()
    
    for i in range(200):
        randomPts = np.array(random.sample(list(matchingPts), 4))
        [T, inliers] = recoverTransformation(randomPts, matchingPts)
        if(inliers.shape[0] > bestInliers.shape[0]):
            bestInliers = inliers.copy()
            bestT = T
    
    for i in range(bestInliers.shape[0]):
        bMatches = cv2.line(bMatches, (int(bestInliers[i][0][0]), int(bestInliers[i][1][0])), (int(I1.shape[1] + bestInliers[i][0][1]), int(bestInliers[i][1][1])), (0, 255, 255), 1)
    
    T_Image = np.array(cv2.warpAffine(I2, bestT, (I1.shape[1]*2, I1.shape[0])))
    
    for i in range(I1.shape[0]):
        for j in range(I1.shape[1]):
            #if(T_Image[i][j][0]==0 and T_Image[i][j][1]==0 and T_Image[i][j][2]==0):
            T_Image[i][j] = I1[i][j]
    
    panoramaWidth = T_Image.shape[1]
    panoramaHeight = T_Image.shape[0]
    
    while(True):
        if(T_Image[int(T_Image.shape[0]/2)][panoramaWidth-1][0]==0 and T_Image[int(T_Image.shape[0]/2)][panoramaWidth-1][1]==0 and T_Image[int(T_Image.shape[0]/2)][panoramaWidth-1][2]==0):
            panoramaWidth=panoramaWidth-1
        else:
            break
    
    while(True):
        if(T_Image[panoramaHeight-1][panoramaWidth-1][0]==0 and T_Image[panoramaHeight-1][panoramaWidth-1][1]==0 and T_Image[panoramaHeight-1][panoramaWidth-1][2]==0):
            panoramaHeight=panoramaHeight-1
        else:
            break
    
    stitchedImage = np.zeros((panoramaHeight, panoramaWidth, T_Image.shape[2]))
    
    for i in range(panoramaHeight):
        for j in range(panoramaWidth):
            stitchedImage[i][j] = T_Image[i][j]
    
    return [featurePoints1, featurePoints2, matches, bMatches, bestT, stitchedImage]

## Running algorithm on Set 2

In [12]:
base_path="Dataset/set2/"
result_path="Results/set2/"

# Extract local feature points for images in the set
Image1 = np.array(cv2.imread(os.path.join(base_path, f"set2_1_100_AML.png")))
Image2 = np.array(cv2.imread(os.path.join(base_path,  f"set2_2_100_AML.png")))
# Include the necessary code here for the functions stitchImages
[featurePoints1, featurePoints2, matches, bestMatches, bestT, stitchedImage] = stitchImages(Image1, Image2)
# Save the results of each step
cv2.imwrite(os.path.join(result_path, "featurePoints1.jpg"), featurePoints1)
cv2.imwrite(os.path.join(result_path, "featurePoints2.jpg"), featurePoints2)
cv2.imwrite(os.path.join(result_path, "matches.jpg"), matches)
cv2.imwrite(os.path.join(result_path, "bestMatches.jpg"), bestMatches)
cv2.imwrite(os.path.join(result_path,  "panorama.jpg"), stitchedImage)


I1.dtype uint8


True

## ## Running algorithm on Natural Set

In [11]:
base_path="Dataset/naturalset/"
result_path="Results/naturalset/"


# Extract local feature points for images in the set
Image1 = np.array(cv2.imread(os.path.join(base_path, f"nature7.jpg")))
Image2 = np.array(cv2.imread(os.path.join(base_path,  f"nature8.jpg")))
# Include the necessary code here for the functions stitchImages
[featurePoints1, featurePoints2, matches, bestMatches, bestT, stitchedImage] = stitchImages(Image1, Image2)
# Save the results of each step
cv2.imwrite(os.path.join(result_path, "naturalset_FeaturePoints1.jpg"), featurePoints1)
cv2.imwrite(os.path.join(result_path, "naturalset_FeaturePoints2.jpg"), featurePoints2)
cv2.imwrite(os.path.join(result_path, "naturalset_matches.jpg"), matches)
cv2.imwrite(os.path.join(result_path, "naturalset_BMatches.jpg"), bestMatches)
cv2.imwrite(os.path.join(result_path,  "naturalset_panorama.jpg"), stitchedImage)

I1.dtype uint8


True